In [39]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [27]:
#For resolution of the plot
plt.rcParams['figure.dpi'] = 200
#For setting background colours
sns.set(rc={'axes.facecolor': '#faded9'}, style='darkgrid')

In [43]:
#    Variable	                 Description
#    age	                         Age of the patient in years
#    sex	                         Gender of the patient (0 = male, 1 = female)
#    cp	                         Chest pain type:
#                                      0: Typical angina
#                                      1: Atypical angina
#                                      2: Non-anginal pain
#                                      3: Asymptomatic
#    trestbps	                 Resting blood pressure in mm Hg
#    chol	                     Serum cholesterol in mg/dl
#    fbs	                         Fasting blood sugar level, categorized as above 120 mg/dl (1 = true, 0 = false)
#    restecg	                     Resting electrocardiographic results:
#                                      0: Normal
#                                      1: Having ST-T wave abnormality
#                                      2: Showing probable or definite left ventricular hypertrophy
#    thalach	                     Maximum heart rate achieved during a stress test
#    exang	                     Exercise-induced angina (1 = yes, 0 = no)
#    oldpeak	                     ST depression induced by exercise relative to rest
#    slope	                     Slope of the peak exercise ST segment:
#                                      0: Upsloping
#                                      1: Flat
#                                      2: Downsloping
#    ca	                         Number of major vessels (0-4) colored by fluoroscopy
#    thal	                     Thalium stress test result:
#                                      0: Normal
#                                      1: Fixed defect
#                                      2: Reversible defect
#                                      3: Not described
#    target	                     Heart disease status (0 = no disease, 1 = presence of disease)"""

In [29]:
#Dataset
df = pd.read_csv("D:\\Project\\heart.csv")
df 

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [31]:
#Display
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [33]:
#Continuous features
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

#Non-continuous featurs
features_to_convert = [feature for feature in df.columns if feature not in continuous_features]

# Coverting it inot object datatype
df[features_to_convert] = df[features_to_convert].astype('object')

df.dtypes

age           int64
sex          object
cp           object
trestbps      int64
chol          int64
fbs          object
restecg      object
thalach       int64
exang        object
oldpeak     float64
slope        object
ca           object
thal         object
target       object
dtype: object

In [35]:
# Description
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,303.0,54.366337,9.082101,29.0,47.5,55.0,61.0,77.0
trestbps,303.0,131.623762,17.538143,94.0,120.0,130.0,140.0,200.0
chol,303.0,246.264026,51.830751,126.0,211.0,240.0,274.5,564.0
thalach,303.0,149.646865,22.905161,71.0,133.5,153.0,166.0,202.0
oldpeak,303.0,1.039604,1.161075,0.0,0.0,0.8,1.6,6.2


In [45]:
# Summary
df.describe(include='object')

,sex,cp,fbs,restecg,exang,slope,ca,thal,target
count,303,303,303,303,303,303,303,303,303
unique,2,4,2,3,2,3,5,4,2
top,1,0,0,1,0,2,0,2,1
freq,207,143,258,152,204,142,175,166,165


In [47]:
# Head
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
